## The Jupyter file will guide you to go through an interesting experiment -- using GNN to achieve a BIM node classification task.

In [1]:
import os 
# torch and dgl
import torch
from torch.utils.data import DataLoader
import torch.nn.functional as F
from dgl.data.utils import load_graphs
# basic machine learning libs
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix

# self construct functions
from node_evaluation import collate, evalEdge 
from SAGEE import SAGEE

pd.options.mode.chained_assignment = None  # default='warn'

device = torch.device('cpu') # CPU is enough for processing small graphs
print('Using device:', device)

Using backend: pytorch


Using device: cpu


Set basic parameters. The default runing epoch is 200. You can play with different hyperparameters. 

In [2]:
epochs = 2
batch_size = 1
n_classes = 9 # nine room classes here
weight_decay=5e-4
num_channels = 50
lr = 0.005

### Load RoomGraph dataset.

RoomGraph is a self-designed graph dataset containing 224 apartment layouts collecting from 3 countries. 

RoomGraph has 9 different node classes, and each node and edge owns its feature matrix.

In [3]:
bg = load_graphs("./../dataset/roomgraph.bin")[0]

# data split
trainvalid, test_dataset =  train_test_split(bg, test_size=0.2, random_state=42)
train_dataset, valid_dataset = train_test_split(trainvalid, test_size=0.1, random_state=42)

# data batch for parallel computation
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, collate_fn=collate)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate)

print("train dataset %i, val dataset %i, test dataset %i"%(len(train_dataset), \
    len(valid_dataset), len(test_dataset)))

train dataset 161, val dataset 18, test dataset 45


### Load SAGE-E model. 

SAGE-E is an improved algorithm based on [GraphSAGE](https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf). 

The main improvement is that SAGE-E can leverage both node and edge features, but GraphSAGE can only learn from node features.

In [4]:
# model loading 
ndim_in = train_dataset[0].ndata['feat'].shape[1]
edim_in = train_dataset[0].edata['relation'].shape[1]

model = SAGEE(ndim_in, n_classes, edim_in,  F.relu, 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
model = model.to(device)
print(model)

SAGEE(
  (layers): ModuleList(
    (0): SAGEELayer(
      (W_msg): Linear(in_features=13, out_features=50, bias=True)
      (W_apply): Linear(in_features=58, out_features=50, bias=True)
    )
    (1): SAGEELayer(
      (W_msg): Linear(in_features=55, out_features=50, bias=True)
      (W_apply): Linear(in_features=100, out_features=50, bias=True)
    )
    (2): SAGEELayer(
      (W_msg): Linear(in_features=55, out_features=25, bias=True)
      (W_apply): Linear(in_features=75, out_features=25, bias=True)
    )
    (3): SAGEELayer(
      (W_msg): Linear(in_features=30, out_features=9, bias=True)
      (W_apply): Linear(in_features=34, out_features=9, bias=True)
    )
  )
  (dropout): Dropout(p=0.2, inplace=False)
)


### Start to train the model

In [5]:
train_acc_all, train_loss_all  = [], []
val_acc_all, val_loss_all = [], []


train_startime = time.time()

for epoch in range(epochs):

    #### train one epoch 
    model.train()

    train_acc_list = []
    train_loss_list = []

    # feed graph to algorithm one by one
    for batch, subgraph in enumerate(train_dataloader):

        subgraph = subgraph.to(device) 
        nfeat = subgraph.ndata['feat'].float()
        efeat = subgraph.edata['relation'].float()

        logits = model(subgraph, nfeat, efeat) # get the prediction from models 

        # calculate the accuracy 
        gt = torch.argmax(subgraph.ndata['label'], dim=1) # ground true labels
        pre  = torch.argmax(logits, dim=1)  # prediction labels 
        correct = torch.sum(pre == gt) # calculate the right labels 

        acc = correct.item()*1.0/len(gt) # calculate the accuracy 
        train_acc_list.append(acc) 

        # compute the loss
        loss = F.cross_entropy(logits, gt) # using cross entropy 
        train_loss_list.append(loss.item()) 

        # backward propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # calculate acc and loss for each epoch
    train_loss_epoch =  np.array(train_loss_list).mean()
    train_acc_epoch =  np.array(train_acc_list).mean()

    train_loss_all.append(train_loss_epoch)
    train_acc_all.append(train_acc_epoch)

    print("Epoch {:03d} train | Accuracy: {:.4f} | Loss: {:.4f}".format(\
        epoch+1, train_acc_epoch, train_loss_epoch))


    #### start evaluation
    val_acc_list, val_loss_list = [], []

    for batch, subgraph in enumerate(valid_dataloader):
        subgraph = subgraph.to(device)

        # calculate the accuracy and loss
        nfeat = subgraph.ndata['feat'].float()
        efeat = subgraph.edata['relation'].float()

        acc, loss, _, _, _, _ = evalEdge(model, nfeat, efeat, subgraph, subgraph.ndata['label'], n_classes)

        # obtain acc and loss
        val_acc_list.append(acc)
        val_loss_list.append(loss.item())

    # calculate the loss and acc for all graphs in one epoch
    val_loss_epoch =  np.array(val_loss_list).mean()
    val_acc_epoch =  np.array(val_acc_list).mean()

    # append for drawing the curs
    val_acc_all.append(val_acc_epoch)
    val_loss_all.append(val_loss_epoch)
        
    print("Validation | Accuracy: {:.4f} | Loss: {:.4f}\n".format(val_acc_epoch, val_loss_epoch))

    ############ save the best acc epoch ############
    if val_acc_epoch >= max(val_acc_all):
        torch.save(model, "best_user.pt")
    
train_endtime = time.time()

print("Finish training! Using {:.4f} s:".format(train_endtime - train_startime))


Epoch 001 train | Accuracy: 0.3210 | Loss: 1.8735
Validation | Accuracy: 0.4354 | Loss: 1.6016

Epoch 002 train | Accuracy: 0.4663 | Loss: 1.5542
Validation | Accuracy: 0.5122 | Loss: 1.3558

Finish training! Using 4.7578 s:


### Start to test 

We prepare our best weight file here. (named as "best.pt")
The test accuracy based on our weight file is around 80%. 

You can also set your criterion to select your best weight, and test here. 

In [9]:
if not os.path.exists("best_user.pt"):
    filename = "best_default.pt"
    print("Load default best weight")
else:
    filename="best_user.pt"
    print("Load user best weight")


model = torch.load(filename)  # read the best weight
model.eval()    

# print(model)

test_acc_list = [] # list for storing the acc from each graph
pre, gt = [], []


test_startime = time.time()

for batch, subgraph in enumerate(test_dataloader):
    subgraph = subgraph.to(device)
    # subgraph = dgl.add_self_loop(subgraph)

    nfeat = subgraph.ndata['feat'].float()
    efeat = subgraph.edata['relation'].float()

    acc, _, _, _, one_pre, one_gt = evalEdge(model, nfeat, efeat, \
        subgraph, subgraph.ndata['label'], n_classes)

    test_acc_list.append(acc)
    pre.extend(one_pre)
    gt.extend(one_gt)



test_time = time.time() - test_startime

test_acc = np.array(test_acc_list).mean()

cm = confusion_matrix(gt, pre)  # confusion matrix, default function from scikit-learn

f1 = f1_score(gt, pre, average='macro') # f1, default function from scikit-learn

print("Test Accuracy: {:.4f}".format(test_acc))

print("F1 score: {:4f}".format(f1))

print("Test time: {:.4f} s".format(test_time))

print(f"Confusion matrix:\n{cm}")

Load default best weight
Test Accuracy: 0.8222
F1 score: 0.809913
Test time: 0.3301 s
Confusion matrix:
[[35  0  9  0  0  0  0  0  1]
 [ 0 45  0  0  0  0  0  0  0]
 [ 7  0 37  0  0  0  0  0  0]
 [ 0  0  0 70  8  3  0  0  0]
 [ 0  0  0  2 29  1  0  0  0]
 [ 1  0  0 11  0 49  5  0  4]
 [ 0  0  0  2  0 11 14  0  0]
 [ 0  0  0  0  0  0  0 41  0]
 [ 0  0  0  4  0  7  0  0 19]]
